# ![](CoronavirusAnimation2.gif)

In [136]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
from datetime import date, timedelta
from sklearn.cluster import KMeans
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import plotly.offline as py
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from keras.models import Sequential
from keras.layers import LSTM,Dense
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator


***Bangladesh Covid-19 Case Summary Representation***

In [139]:
dataFromCsv = pd.read_csv('covid-19-PerDayCaseSummaryBD.csv')
totalCaseNumber = dataFromCsv['Case'].sum()
totalDeathNumber = dataFromCsv['Death'].sum()
totalRecoveredNumber = dataFromCsv['Recovered'].sum()
totalPatientNumber = totalCaseNumber - (totalDeathNumber+totalRecoveredNumber)

slices = [totalPatientNumber,totalDeathNumber,totalRecoveredNumber]
activities = ['Patient','Death','Recovered']

dataFrame = {
    "slices" : slices,
    "activities" : activities
}
dataset = pd.DataFrame(dataFrame)
fig = px.pie(dataset,
             values="slices",
             names="activities",
             template="presentation",
             labels = {'slices' : 'No Cases', 'activities' : 'Status'},
             color_discrete_sequence=['#4169E1', '#DC143C', '#006400'],
             width=800,
             height=450,
             hole=0.6,
             title="Covid-19 Case Summary BD")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
import plotly.offline as py
py.offline.iplot(fig)

In [140]:
dataFromCsv = pd.read_csv('covid-19-PerDayCaseSummaryBD.csv')
totalCaseNumber = int(dataFromCsv['Case'].iloc[-1])
totalDeathNumber = int(dataFromCsv['Death'].iloc[-1])
totalRecoveredNumber = int(dataFromCsv['Recovered'].iloc[-1])
totalPatientNumber = totalCaseNumber - (totalDeathNumber+totalRecoveredNumber)

slices = [totalPatientNumber,totalDeathNumber,totalRecoveredNumber]
activities = ['Patient','Death','Recovered']

dataFrame = {
    "slices" : slices,
    "activities" : activities
}
dataset = pd.DataFrame(dataFrame)
fig = px.pie(dataset,
             values="slices",
             names="activities",
             template="ggplot2",
             labels = {'slices' : 'No Cases', 'activities' : 'Status'},
             color_discrete_sequence=['#4169E1', '#006400', '#DC143C'],
             width=800,
             height=450,
             hole=0.6,
             title="Covid-19 Case Summary 24 Hours")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()

In [134]:
# dataFromCsv = pd.read_csv('../Analysis Data/covid-19-PerDayCaseSummaryBD.csv')
dataset = pd.read_csv('covid-19-PerDayCaseSummaryBD.csv',parse_dates=["Date"], index_col='Date')
dataset = dataset.cumsum()
dataset = dataset.reset_index()
datetimeobject = datetime.datetime.strptime(str(dataFromCsv['Date'].iloc[1]),'%m/%d/%Y')
startDate = datetimeobject.strftime('%d-%b-%Y')
datetimeobject = datetime.datetime.strptime(str(dataFromCsv['Date'].iloc[-1]),'%m/%d/%Y')
endDate = datetimeobject.strftime('%d-%b-%Y')
xlabel = str(startDate) + ' UPTO ' + str(endDate)
fig = px.area(dataset, x="Date", y="Case", 
              template="presentation", 
              labels = {'Date': xlabel, 'Case': 'Positive Cases'},
              title='Postive Case Trend',
              color_discrete_sequence=['#4169E1'])
fig2 = px.area(dataset, x="Date", y="Death", 
               template="presentation",
               labels = {'Date': xlabel, 'Death': 'Death Cases'},
               title='Death Case Trend',
               color_discrete_sequence=['#DC143C'])

fig3 = px.area(dataset, x="Date", y="Recovered", 
               template="presentation",
               labels = {'Date': xlabel, 'Recovered': 'Recovered Cases'},
               title='Recovered Case Trend',
               color_discrete_sequence=['#006400'])
fig.update_traces(textposition='top left',mode='lines+markers')
fig2.update_traces(textposition='top left',mode='lines+markers')
fig3.update_traces(textposition='top left',mode='lines+markers')
fig.show()
fig2.show()
fig3.show()

In [141]:
import plotly.graph_objects as go
dataset = pd.read_csv('covid-19-PerDayCaseSummaryBD.csv',parse_dates=["Date"])
dataset = dataset.reset_index()
animals=['giraffes', 'orangutans', 'monkeys']

fig = go.Figure(data=[
    go.Bar(name='Positove', x=dataset['Date'], y=dataset['Case'], marker_color = '#4169E1'),
    go.Bar(name='Death', x=dataset['Date'], y=dataset['Death'], marker_color = '#DC143C'),
    go.Bar(name='Recovered', x=dataset['Date'], y=dataset['Recovered'], marker_color = '#006400')
])
# Change the bar mode
fig.update_layout(barmode='stack',template="presentation",title_text='Per Day Case Summary')
fig.show()

In [142]:
symptoms={'symptom':['Fever',
        'Dry cough',
        'Fatigue',
        'Sputum production',
        'Shortness of breath',
        'Muscle pain',
        'Sore throat',
        'Headache',
        'Chills',
        'Nausea or vomiting',
        'Nasal congestion',
        'Diarrhoea',
        'Haemoptysis',
        'Conjunctival congestion'],'percentage':[87.9,67.7,38.1,33.4,18.6,14.8,13.9,13.6,11.4,5.0,4.8,3.7,0.9,0.8]}

symptoms=pd.DataFrame(data=symptoms,index=range(14))
symptoms

,symptom,percentage
0,Fever,87.9
1,Dry cough,67.7
2,Fatigue,38.1
3,Sputum production,33.4
4,Shortness of breath,18.6
5,Muscle pain,14.8
6,Sore throat,13.9
7,Headache,13.6
8,Chills,11.4
9,Nausea or vomiting,5.0


In [144]:
fig = px.bar(symptoms[['symptom', 'percentage']].sort_values('percentage', ascending=False), 
             y="percentage", x="symptom", color='symptom', 
             log_y=True, template='ggplot2', title='Symptom of  Coronavirus')
fig.show()

In [145]:
fig = px.pie(symptoms,
             values="percentage",
             names="symptom",
             template="seaborn")
fig.update_traces(rotation=90, pull=0.05, textinfo="percent+label")
fig.show()